## Examples Exporting Parquet to SQL

In [ ]:
from os import environ
import urllib

import pyarrow.parquet as pq
from sqlalchemy import create_engine


In [ ]:
%%bash

ls -al

## Setup SQL Engine

In [ ]:
driver = environ.get('SQL_DRIVER', '{ODBC Driver 17 for SQL Server}')
host = environ.get('SQL_HOST', 'sql-jupyter-lib')
db = environ.get('SQL_DB', 'ScratchDB')
user = environ.get('SQL_USER', 'sa')
pw = environ.get('SQL_PASSWORD', 'HelloWorld1')
con_str = f'DRIVER={driver};SERVER={host};DATABASE={db};UID={user};PWD={pw}'

params = urllib.parse.quote_plus(con_str)
sql_engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}"
                           , echo=False) # echo's emitted sql

## Export Parquet Datasets

In [ ]:
%%time

with sql_engine.connect() as c:
    datasets = [
        ('KinnserBIBaseData', 'data/KinnsrBIBaseData'),
        ('KinnserCrosswalk', 'data/cc_crosswalk_kinnser'),
        ('KinnserPatientScore', 'data/patient_score_kinnser'),
        ('LTC400BaseData', 'data/LTC400BaseData'),
        ('LTC400Crosswalk', 'data/cc_crosswalk_ltc400'),
        ('LTC400PatientScore', 'data/patient_score_ltc400'),
    ]
    for table, folder in datasets:
        print(f"Reading from {folder}")

        dataset = pq.ParquetDataset(folder)
        pq_table = dataset.read()
        df = pq_table.to_pandas()

        print(f"Loading to {table}")
        df.to_sql(name=table, con=c, schema='Staging', 
                  if_exists='append', index=True)